In [1]:
#1 - Import libraries
import requests
import pandas as pd
import time

In [2]:
#2 - Keys
API_KEY = ""
#CHANNEL_ID = "UCX6OQ3DkcsbYNE6H8uQQuVA"             # Mr Beast
#CHANNEL_ID = "UC-lHJZR3Gqxm24_Vd_AJ5Yw"             # PewDiePie
CHANNEL_ID = "UCINb0wqPz-A0dV9nARjJlOQ"             # The Dodo

In [3]:
#3 - Calling API
pageToken = ""

#from video
url = "https://www.googleapis.com/youtube/v3/search?key=" + API_KEY + "&channelId=" + CHANNEL_ID + "&part=snippet,id&order=date&maxResults=10000&" + pageToken
response = requests.get(url).json()
#response           # uncomment to see raw json file

In [4]:
#4 - Display first item (latest upload)
response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': '8SZA1sL_78bel95Zr3_eAfXSc-s',
 'id': {'kind': 'youtube#video', 'videoId': '56AfIj3ij70'},
 'snippet': {'publishedAt': '2022-07-16T17:00:06Z',
  'channelId': 'UCINb0wqPz-A0dV9nARjJlOQ',
  'title': 'Hearbroken Vulture Can Finally Fly Again Thanks To His Rescuers | The Dodo',
  'description': 'This Indian vulture was in search of water in the Summer heat of Nashik, India when it got so dehydrated, he fell to the ground.',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/56AfIj3ij70/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/56AfIj3ij70/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/56AfIj3ij70/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'The Dodo',
  'liveBroadcastContent': 'none',
  'publishTime': '2022-07-16T17:00:06Z'}}

In [5]:
# Refer to #4 (grab value of videoId from key 'items')
# Comment out (optional)
video_id = response['items'][0]['id']['videoId']
video_id

'56AfIj3ij70'

In [6]:
# Refer to #4 (grab value of title from key 'items')
# Remove excess characters with replace
# Comment out (optional)
video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace("&amp;","")
video_title

'Hearbroken Vulture Can Finally Fly Again Thanks To His Rescuers | The Dodo'

In [7]:
# Refer to #4 (grab value of publishedAt from key 'items')
# Remove all characters to the right after specified split character
# Comment out (optional)
upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split("T")[0]
upload_date

'2022-07-16'

In [8]:
# 2nd API Call / video_id is called in video for loop
def get_video_information(video_id):
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id=" + video_id + "&part=statistics&key=" + API_KEY

    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, comment_count

In [9]:
# Passing Pandas dataframe into function
def get_videos(df):
    # These 3 lines are from Cell # 3
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key=" + API_KEY + "&channelId=" + CHANNEL_ID + "&part=snippet,id&order=date&maxResults=10000&" + pageToken
    response = requests.get(url).json()

    time.sleep(1)       # 1 second delay / provides API enough time to process

    # For loop that grabs info specifically from videos
    for video in response['items']:
        if video['id']['kind'] == "youtube#video":
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            video_title = str(video_title).replace("&amp;","")
            video_title = str(video_title).replace("&#39;","'")         # can remove b/c added (encoding='utf-8-sig') to df.to_csv 
            video_title = str(video_title).replace("â€™","'")           # can remove b/c added (encoding='utf-8-sig') to df.to_csv 
            upload_date = video['snippet']['publishedAt']
            upload_date = str(upload_date).split("T")[0]

            # 2nd API call gets called here
            view_count, like_count, comment_count = get_video_information(video_id)


            # Saving data into Pandas dataframe with append
            df = df.append({
                'video_id': video_id,
                'video_title': video_title,
                'upload_date': upload_date,
                'view_count': view_count,
                'like_count': like_count,
                'comment_count': comment_count}, ignore_index=True)


            # Outputs
            # print(video_id)
            # print(video_title)
            # print(upload_date)
            # print(view_count)
            # print(like_count)
            # print(comment_count)

    return df

In [10]:
# Build dataframe (built empty / populated after for loop)
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"])

df = get_videos(df)

In [11]:
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,56AfIj3ij70,Hearbroken Vulture Can Finally Fly Again Thank...,2022-07-16,111500,9929,515
1,tk9GsEjoWhk,Guy Has Met 150 Cats Walking Through His Neigh...,2022-07-16,152904,12005,705
2,GrpBvbmFaAE,Cats Become Obsessed With Their Mom's New Vita...,2022-07-15,55371,3770,82
3,eNVPwmmoRJQ,Tiny Sick Kitten Sits On Volunteer's Shoulder ...,2022-07-15,193323,12912,523
4,fbE8KB-8Y7Y,Rescue Pit Bull Loves The Drive Thru At McDona...,2022-07-15,282872,15761,762
5,88Gn2E3Dhpk,Guy Saves Bald Eagle From Drowning In River | ...,2022-07-14,882034,31593,1711
6,GS-L47SssoM,Cat Became So Scared After This Huge Animal St...,2022-07-14,298652,10996,467
7,zmdUTlPf1sU,Cat Will Not Begin His Day If His Mom Doesn't ...,2022-07-13,233118,17378,408
8,mnL0zWbG2yA,Smiling Pittie Goes To The Pier Everyday To Vi...,2022-07-13,484604,20726,1036
9,CCIamKjIJkc,The Survival Guide To: Getting A Younger Sibli...,2022-07-12,200649,10326,392


In [12]:
df.to_csv('youtube_channel-The_Dodo.csv',encoding='utf-8-sig')